In [58]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder
from scipy.stats import randint
import numpy as np

# Load datasets
df_train = pd.read_csv("EPL_assignment/epl_matches_train.csv")
df_test = pd.read_csv("EPL_assignment/epl_matches_test.csv")
df_teams = pd.read_csv("EPL_assignment/epl_teams.csv")
df_players = pd.read_csv("EPL_assignment/epl_players.csv")
df_goals = pd.read_csv("EPL_assignment/epl_goals.csv")

# Ensure no missing values
df_train.dropna(inplace=True)
df_test.dropna(inplace=True)
df_teams.dropna(inplace=True)
df_players.dropna(inplace=True)
df_goals.dropna(inplace=True)

# Create target variable 'match_result'
def get_match_result(home_goals, away_goals):
    if home_goals > away_goals:
        return 1
    elif home_goals < away_goals:
        return -1
    else:
        return 0

df_train["match_result"] = df_train.apply(lambda x: get_match_result(x["home_team_goal"], x["away_team_goal"]), axis=1)

# Convert date columns to datetime format
df_teams["date"] = pd.to_datetime(df_teams["date"])
df_players["date"] = pd.to_datetime(df_players["date"])



### Merge Team & Player Data

In [59]:
df_train = df_train.merge(df_teams, left_on="home_team_id", right_on="team_id", suffixes=("", "_home"))
df_train = df_train.merge(df_teams, left_on="away_team_id", right_on="team_id", suffixes=("", "_away"))
df_test = df_test.merge(df_teams, left_on="home_team_id", right_on="team_id", suffixes=("", "_home"))
df_test = df_test.merge(df_teams, left_on="away_team_id", right_on="team_id", suffixes=("", "_away"))

# Drop redundant team_id columns
df_train.drop(columns=["team_id", "team_id_away"], inplace=True)
df_test.drop(columns=["team_id", "team_id_away"], inplace=True)

# Merge Player Data - Compute Average Skill per Team
player_stats = ["dribbling", "finishing", "short_passing", "ball_control", "acceleration", "stamina"]
df_players_avg = df_players.groupby("player_id")[player_stats].mean().reset_index()

def get_team_avg(df, prefix):
    player_columns = [f"{prefix}_player_{i}" for i in range(1, 12)]
    team_avg = df[player_columns].apply(lambda x: df_players_avg[df_players_avg["player_id"].isin(x)].mean(), axis=1)
    return team_avg[player_stats]

df_train[player_stats] = get_team_avg(df_train, "home")
df_train[[f"{stat}_away" for stat in player_stats]] = get_team_avg(df_train, "away")
df_test[player_stats] = get_team_avg(df_test, "home")
df_test[[f"{stat}_away" for stat in player_stats]] = get_team_avg(df_test, "away")

# Drop player ID columns
df_train.drop(columns=[f"home_player_{i}" for i in range(1, 12)] + [f"away_player_{i}" for i in range(1, 12)], inplace=True)
df_test.drop(columns=[f"home_player_{i}" for i in range(1, 12)] + [f"away_player_{i}" for i in range(1, 12)], inplace=True)


### Merge Goals Data

In [60]:
df_goals_agg = df_goals.groupby(["match_id", "team_id"])["goal_number"].count().reset_index()
df_goals_agg.rename(columns={"goal_number": "total_goals"}, inplace=True)
df_goals_agg.rename(columns={"team_id": "goal_team_id"}, inplace=True)

# Merge goals data into training and test sets, ensuring no duplication of team_id
df_train = df_train.merge(df_goals_agg, left_on=["match_id", "home_team_id"], right_on=["match_id", "goal_team_id"], how="left").rename(columns={"total_goals": "total_goals_home"})
df_train = df_train.merge(df_goals_agg, left_on=["match_id", "away_team_id"], right_on=["match_id", "goal_team_id"], how="left").rename(columns={"total_goals": "total_goals_away"})

df_test = df_test.merge(df_goals_agg, left_on=["match_id", "home_team_id"], right_on=["match_id", "goal_team_id"], how="left").rename(columns={"total_goals": "total_goals_home"})
df_test = df_test.merge(df_goals_agg, left_on=["match_id", "away_team_id"], right_on=["match_id", "goal_team_id"], how="left").rename(columns={"total_goals": "total_goals_away"})

# Fill NaNs with 0
df_train.fillna(0, inplace=True)
df_test.fillna(0, inplace=True)

### Train Model & Predict

In [61]:
# Select only numeric columns (int64, float64)
df_train_numeric = df_train.select_dtypes(include=['number'])
df_test_numeric = df_test.select_dtypes(include=['number'])

# Now df_train_numeric and df_test_numeric contains only numeric columns

# Get the columns from df_test
df_test_columns = df_test_numeric.columns

# Filter df_train to only include columns that are also in df_test, plus "match_result"
df_train_filtered = df_train_numeric[df_test_columns.tolist() + ['match_result']]

In [62]:
df_train_filtered.drop(columns=["total_goals_home", "total_goals_away"])

,stage,match_id,home_team_id,away_team_id,home_player_X1,home_player_X2,home_player_X3,home_player_X4,home_player_X5,home_player_X6,...,stamina,dribbling_away,finishing_away,short_passing_away,ball_control_away,acceleration_away,stamina_away,goal_team_id_x,goal_team_id_y,match_result
0,1,49337,10260,10261,1,2,4,6,8,2,...,68.846775,64.926994,52.806998,68.965254,68.965347,70.920863,70.398923,10260.0,10261.0,0
1,1,49337,10260,10261,1,2,4,6,8,2,...,68.846775,64.926994,52.806998,68.965254,68.965347,70.920863,70.398923,10260.0,10261.0,0
2,1,49337,10260,10261,1,2,4,6,8,2,...,68.846775,64.926994,52.806998,68.965254,68.965347,70.920863,70.398923,10260.0,10261.0,0
3,1,49337,10260,10261,1,2,4,6,8,2,...,68.846775,64.926994,52.806998,68.965254,68.965347,70.920863,70.398923,10260.0,10261.0,0
4,1,49586,10260,10261,1,2,4,6,8,2,...,72.552566,61.314710,53.658249,65.480766,67.209205,64.626747,71.944900,10260.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10311,12,53276,8344,10260,1,2,4,6,8,4,...,71.932966,64.594316,51.568710,70.694443,70.308630,72.776439,68.147364,8344.0,10260.0,0
10312,5,38237,8197,10260,1,2,4,6,8,4,...,70.508262,64.217320,57.428851,71.105889,71.807094,71.764227,68.532923,8197.0,10260.0,1
10313,5,38237,8197,10260,1,2,4,6,8,4,...,70.508262,64.217320,57.428851,71.105889,71.807094,71.764227,68.532923,8197.0,10260.0,1
10314,5,38237,8197,10260,1,2,4,6,8,4,...,70.508262,64.217320,57.428851,71.105889,71.807094,71.764227,68.532923,8197.0,10260.0,1


In [63]:
# Now df_train_filtered will have only the columns that exist in both df_train and df_test
X = df_train_filtered.drop(columns=["match_result"])
y = df_train_filtered["match_result"]

# Encode target variable
label_encoder = LabelEncoder()
label_encoder.fit(y)  # Fit with all possible labels
y = label_encoder.transform(y)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=30, stratify=y)

In [64]:
# Define parameter grid for RandomizedSearchCV
param_dist = {
    'n_estimators': randint(100, 500),
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(1, 10),
    'bootstrap': [True, False],
    'class_weight': [None, "balanced", "balanced_subsample"]
}

clf = RandomForestClassifier(random_state=25)
random_search = RandomizedSearchCV(RandomForestClassifier(random_state=25),
                                   param_distributions=param_dist, 
                                   n_iter=30, cv=5, n_jobs=-1, random_state=42)

random_search.fit(X_train, y_train)

# Best model predictions
y_proba = random_search.best_estimator_.predict_proba(df_test_numeric)

In [65]:
# Custom threshold (e.g., if draw probability < 40%, assign Win/Lose)
threshold = 0.4  
predicted_class = []
for probs in y_proba:
    if probs[1] > threshold:
        predicted_class.append(0)  # Draw
    else:
        predicted_class.append(np.argmax(probs))  # Assign Win/Loss based on higher probability


df_test_numeric["predicted_match_result"] = label_encoder.inverse_transform(predicted_class)

# Define the mapping
result_mapping = {"Win": 1, "Lose": -1, "Draw": 0}

# Replace the predicted values using the mapping
df_test_numeric["predicted_match_result"] = df_test_numeric["predicted_match_result"].map({v: k for k, v in result_mapping.items()})

# Remove all columns that have 0.0 in all rows
df_test_numeric = df_test_numeric.loc[:, (df_test_numeric != 0).any(axis=0)]

df_test_numeric = df_test_numeric.drop_duplicates(subset=["match_id"])

# Save results to CSV
df_test_numeric[["home_team_id", "away_team_id", "predicted_match_result"]].to_csv("predicted_results_v3.csv", index=False)

In [66]:
df_test_numeric

,stage,match_id,home_team_id,away_team_id,home_player_X1,home_player_X2,home_player_X3,home_player_X4,home_player_X5,home_player_X6,...,ball_control,acceleration,stamina,dribbling_away,finishing_away,short_passing_away,ball_control_away,acceleration_away,stamina_away,predicted_match_result
0,1,48994,9825,8654,1,2,4,6,8,4,...,71.781134,69.287084,72.699193,56.473961,47.973520,60.887776,62.759218,69.476046,70.407761,Win
4,21,50862,8678,8654,1,2,4,6,8,5,...,61.858583,68.969614,69.184522,58.678734,50.918094,62.339848,63.322686,69.323769,70.621599,Lose
8,31,59732,8455,8654,1,2,4,6,8,4,...,69.842840,72.376711,74.686034,61.492173,54.461662,61.961752,63.896936,72.762562,71.305716,Win
12,29,53740,8668,8654,1,3,5,7,2,4,...,69.225222,75.384411,73.003802,61.779959,51.219040,64.145749,65.531941,72.465599,71.815615,Draw
16,34,35023,8197,8654,1,2,4,6,8,2,...,64.464613,70.156848,74.618892,61.982714,53.141708,62.650845,65.105219,72.671463,72.492139,Lose
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1488,37,32629,8456,9825,1,2,4,6,8,2,...,71.169551,76.886139,73.540555,64.585254,55.240780,67.056401,67.724915,73.015279,72.028015,Win
1492,18,55795,8466,9825,1,2,4,6,8,4,...,67.326384,70.977581,70.642919,63.682859,53.914192,70.374534,70.032942,71.857374,71.636942,Draw
1496,11,36880,10003,9825,1,2,4,6,8,4,...,66.722090,69.510339,67.916083,64.564744,52.912074,70.509631,70.718241,71.678257,69.689844,Lose
1500,29,55462,8586,9825,1,2,4,6,8,4,...,68.742691,71.825929,71.279398,65.398332,54.170540,70.651169,70.278507,70.046549,71.878901,Lose
